In [2087]:
import random 

test = 1

if test == 0:
    lines = [
        "3 6 7 3 1",
        "0 0 1",
        "1 1 3",
        "2 2 1",
        "1 0 3",
        "0 2 3",
        "2 1 1",
    ]
elif test == 1:
    lines = [
        "3 6 7 3 1",
        "0 0 3",
        "1 1 3",
        "2 2 3",
        "0 1 3",
        "1 2 1",
        "2 0 1",
    ]
elif test == 2:
    lines = [
        "2 2 7 5 3",
        "0 0 3",
        "1 1 5",   
    ]

In [2088]:
def main():
    return 
main()


In [2089]:
""" READ LINES """
# parameters = input()
parameters = lines[0]
numbers = parameters.split(' ')

n = int(numbers[0]) # Volunteer & Cities
m = int(numbers[1]) # Preferences
b = int(numbers[2]) # Badget
t = int(numbers[3]) # Cost Train
c = int(numbers[4]) # Cost Car

VXC = []
for i in range(n): 
    VXC.append([])
    for j in range(n):
            VXC[i].append(-1) 

for idx in range(1,m+1): 
    # preference = input()
    preference = lines[idx]
    numbers = preference.split(' ')
    i = int(numbers[0])
    j = int(numbers[1])
    cost = int(numbers[2])
    VXC[i][j] = cost

print(f"{ n = }")
print(f"{ m = }")
print(f"{ b = }")
print(f"{ t = }")
print(f"{ c = }")
print(f"{ VXC = }")

 n = 3
 m = 6
 b = 7
 t = 3
 c = 1
 VXC = [[3, 3, -1], [-1, 3, 1], [1, -1, 3]]


In [2090]:
""" INITIAL CHECK """
if (
    (m < n) or 
    (t* n < b) or 
    (t % 2 == 0 and c % 2 == 0 and b % 2 == 1)  # t & c even & b odd -> no
): 
    print("no")
    #return

In [2091]:
def is_prime(num):
    if num <= 1:
        return False
    if num <= 3:
        return True  # 2 and 3 are prime
    if num % 2 == 0 or num % 3 == 0:
        return False
    # Check divisors from 5 to sqrt(num)
    i = 5
    while i * i <= num:
        if num % i == 0 or num % (i + 2) == 0:
            return False
        i += 6
    return True

def next_prime(n):
    candidate = n + 1
    while not is_prime(candidate):
        candidate += 1
    return candidate

In [2092]:
size_max = max(t*n, n*n)
size_p = next_prime(size_max)
size_p

11

In [2093]:
F = list(range(size_p))
F

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [2094]:
alpha = []
for i in range(n): 
    alpha.append(random.choices(F, k=n))
    
alpha

[[5, 10, 2], [0, 8, 9], [2, 3, 1]]

In [2095]:
Y = random.sample(F, n*t + 1)
Y

[2, 7, 3, 10, 0, 1, 9, 8, 4, 5]

In [2096]:
def compute_h(y, x, n, VXC, p):
    H = []

    for i in range(n):
        H.append([])
        for j in range(n):
            wij = VXC[i][j]
            xij = x[i][j]
            if wij == -1:
                H[i].append(0)
            else: # a -> b
                H[i].append(((y**wij) * xij) % p)
            
    return H
                
def determinant(matrix):
    # Base case: if the matrix is 1x1, return the single element
    if len(matrix) == 1:
        return matrix[0][0]
    
    # Base case: if the matrix is 2x2, return ad - bc
    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]

    # Recursive case: expand along the first row
    det = 0
    for col in range(len(matrix)):
        # Create a submatrix by excluding the current row and column
        submatrix = [row[:col] + row[col+1:] for row in matrix[1:]]
        # Add or subtract the determinant of the submatrix
        det += ((-1) ** col) * matrix[0][col] * determinant(submatrix)
    
    return det

In [2097]:
s = []
for y in Y: 
    H = compute_h(y, alpha, n, VXC, size_p)
    s.append(determinant(H) % size_p)
s

[5, 8, 10, 0, 0, 0, 6, 1, 3, 1]

In [2098]:
P = []
for j, y in enumerate(Y):
    P.append([])
    for i in range(len(Y)):
        P[j].append((y**i) % size_p)
P

[[1, 2, 4, 8, 5, 10, 9, 7, 3, 6],
 [1, 7, 5, 2, 3, 10, 4, 6, 9, 8],
 [1, 3, 9, 5, 4, 1, 3, 9, 5, 4],
 [1, 10, 1, 10, 1, 10, 1, 10, 1, 10],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 9, 4, 3, 5, 1, 9, 4, 3, 5],
 [1, 8, 9, 6, 4, 10, 3, 2, 5, 7],
 [1, 4, 5, 9, 3, 1, 4, 5, 9, 3],
 [1, 5, 3, 4, 9, 1, 5, 3, 4, 9]]

In [2099]:
def gaussian_elimination(A, b, p):
    n = len(b)
    # Forward elimination
    for i in range(n):
        # Pivoting
        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]
            b[j] -= factor * b[i]

    # Back substitution
    x = [0] * n
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] / A[i][i]) % p
        for j in range(i - 1, -1, -1):
            b[j] -= A[j][i] * x[i]
    return x

In [2100]:
c = gaussian_elimination(P, s, size_p)
c

[7.212658227848124,
 2.1037543220957957,
 4.780803261426875,
 9.886897640580905,
 2.947077707487523,
 9.823529411764719,
 1.8025210084033465,
 0.04621848739496822,
 0.13865546218487557,
 3.0714285714285356]

In [2101]:
has = (c[b+1] % size_p) != 0

In [2102]:
print("yes" if has else "no")

yes


In [1974]:
test = 0